In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdMolTransforms import ComputeCentroid
from vina import Vina
import pdbtools
import subprocess

In [2]:
import pandas as pd 

In [3]:
df = pd.read_csv("../files/initial_binders.csv")

In [4]:
df

,Chembl_id,Similarity,canonical_smiles
0,CHEMBL126,100.000000,CC(=O)NC[C@H]1CN(c2ccc(N3CCOCC3)c(F)c2)C(=O)O1
1,CHEMBL191283,100.000000,CC(=O)NCC1CN(c2ccc(N3CCOCC3)c(F)c2)C(=O)O1
2,CHEMBL7429,100.000000,CC(=O)NC[C@@H]1CN(c2ccc(N3CCOCC3)c(F)c2)C(=O)O1
3,CHEMBL456501,92.592591,CC(=O)NC[C@H]1CN(c2ccc(N3CCCOCC3)c(F)c2)C(=O)O1
4,CHEMBL440011,83.636361,CC(=O)NC[C@H]1CN(c2ccc(N3CCN(c4ccc(F)cc4)CC3)c...
5,CHEMBL39402,83.050847,CC(=O)NC[C@H]1CN(c2ccn(-c3ccc(N4CCOCC4)c(F)c3)...
6,CHEMBL352626,82.142860,CC(=O)NC[C@H]1CN(c2ccc(N3CCN(c4ccccc4)CC3)c(F)...
7,CHEMBL1651124,80.357140,CC(=O)NC[C@H]1CN(c2ccc(N3CCNCC3)c(F)c2)C(=O)O1
8,CHEMBL288149,80.357140,CC(=O)NC[C@H]1CN(c2ccc(N3CCSCC3)c(F)c2)C(=O)O1
9,CHEMBL4460889,100.000000,CC(=O)NC[C@H]1CN(c2ccc(N3CCO[C@@]4(CCCOC4)C3)c...


In [5]:
def convert_smiles_to_mol(x):
    mol1 = Chem.MolFromSmiles(x)
    mol1 = Chem.AddHs(mol1)
    AllChem.EmbedMolecule(mol1)
    AllChem.MMFFOptimizeMolecule(mol1)
    return mol1 

### get centroid from malonate ion 

In [6]:
mli = Chem.MolFromPDBFile("../files/malonate.pdb")
x,y,z = ComputeCentroid(mli.GetConformer())

### prepare vina 

In [7]:
v = Vina(sf_name='vina', seed=101)
v.set_receptor('../files/receptor_docking.pdbqt')
v.compute_vina_maps(center=[x,y,z], box_size=[20, 20, 20])

Computing Vina grid ... done.


In [11]:
energies = []
for Chembl_id,smiles in zip(df["Chembl_id"].tolist(),df["canonical_smiles"].tolist()):
    mol2 = convert_smiles_to_mol(smiles)
    Chem.rdmolfiles.MolToPDBFile(mol2,"../files/pdb_ligand.pdb")
    subprocess.call(f'mk_prepare_ligand.py -i ../files/pdb_ligand.pdb -o ../files/pdb_ligand_prepared.pdbqt --add_hydrogen --pH 7.4 > /dev/null', shell=True)
    ## set ligand 
    v.set_ligand_from_file('../files/pdb_ligand_prepared.pdbqt')
    ## get energies 
    energy = v.score()
    energy_minimized = v.optimize()
    energies.append( (Chembl_id, energy[0], energy_minimized[0] )  ) 
    
    v.write_pose(f'../files/ligand_{Chembl_id}.pdbqt', overwrite=True)

    # Dock the ligand
    v.dock(exhaustiveness=100, n_poses=100)
    v.write_poses(f'../files/ligand_vina_out_{Chembl_id}.pdbqt', n_poses=20, overwrite=True)

Performing local search ... done.
Performing docking (random seed: 101) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1       -6.171          0          0
   2       -6.114          8      11.52
   3       -5.994      3.361      5.249
   4       -5.966      7.423      11.48
   5       -5.908      1.561      1.759
   6       -5.908      7.152       8.07
   7       -5.847      7.303      10.19
   8       -5.838      6.915       7.86
   9       -5.814      7.756      11.51
  10       -5.778      7.407      9.424
  11       -5.771      7.791      10.81
  12        -5.75      8.181      11.24
  13       -5.745      7.491      9.909
  14       -5.721      1.086      1.457
  15        -5.71      7.256       10.6
  16       -5.661      7.245      8.929
 

In [27]:
energy_list = [ (v[0],w[0]) for u,v,w in energies ]

In [29]:
df  = pd. concat ( [ df , pd.DataFrame(energy_list)] , axis=1) 

In [30]:
df.columns

Index(['Chembl_id', 'Similarity', 'canonical_smiles', 0, 1], dtype='object')

In [31]:
df.columns = ['Chembl_id', 'Similarity', 'canonical_smiles', 'vina_score_before_min', 'Vina_score_after_min']

In [34]:
df['Energy_diff'] = df['vina_score_before_min'] - df['Vina_score_after_min']

In [35]:
df

,Chembl_id,Similarity,canonical_smiles,vina_score_before_min,Vina_score_after_min,Energy_diff
0,CHEMBL126,100.000000,CC(=O)NC[C@H]1CN(c2ccc(N3CCOCC3)c(F)c2)C(=O)O1,2.384672e+09,8.552360e+08,1.529436e+09
1,CHEMBL191283,100.000000,CC(=O)NCC1CN(c2ccc(N3CCOCC3)c(F)c2)C(=O)O1,2.384786e+09,2.384786e+09,0.000000e+00
2,CHEMBL7429,100.000000,CC(=O)NC[C@@H]1CN(c2ccc(N3CCOCC3)c(F)c2)C(=O)O1,2.379622e+09,1.404892e+09,9.747305e+08
3,CHEMBL456501,92.592591,CC(=O)NC[C@H]1CN(c2ccc(N3CCCOCC3)c(F)c2)C(=O)O1,2.487000e+09,1.650726e+09,8.362748e+08
4,CHEMBL440011,83.636361,CC(=O)NC[C@H]1CN(c2ccc(N3CCN(c4ccc(F)cc4)CC3)c...,2.937207e+09,2.937207e+09,0.000000e+00
5,CHEMBL39402,83.050847,CC(=O)NC[C@H]1CN(c2ccn(-c3ccc(N4CCOCC4)c(F)c3)...,2.746528e+09,2.746528e+09,0.000000e+00
6,CHEMBL352626,82.142860,CC(=O)NC[C@H]1CN(c2ccc(N3CCN(c4ccccc4)CC3)c(F)...,2.840325e+09,2.120077e+09,7.202477e+08
7,CHEMBL1651124,80.357140,CC(=O)NC[C@H]1CN(c2ccc(N3CCNCC3)c(F)c2)C(=O)O1,2.381719e+09,1.466694e+09,9.150248e+08
8,CHEMBL288149,80.357140,CC(=O)NC[C@H]1CN(c2ccc(N3CCSCC3)c(F)c2)C(=O)O1,2.380410e+09,3.860478e+08,1.994362e+09
9,CHEMBL4460889,100.000000,CC(=O)NC[C@H]1CN(c2ccc(N3CCO[C@@]4(CCCOC4)C3)c...,2.871444e+09,1.693090e+02,2.871444e+09


In [41]:
!du -h ../files//

3.5M	../files/.ipynb_checkpoints
9.6M	../files/


In [39]:
mol_x  = Chem.MolFromPDBFile("../files/receptor_docking.pdbqt")

In [48]:
type(mol_x)

NoneType

In [44]:
mol_y = Chem.MolFromPDBFile("../files/ligand_CHEMBL126.pdbqt")

In [47]:
type(mol_y)

NoneType